In [ ]:
!pip install langchain langchain-groq langchain-community chromadb pandas --quiet

from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
import pandas as pd
import uuid
import chromadb

chroma_client = chromadb.Client()

llm = ChatGroq(
    temperature=0,
    groq_api_key="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",  
    model_name="llama3-70b-8192" 
)

url = "https://atlassian.com/company/careers/details/19715"  
loader = WebBaseLoader(url)
page_data = loader.load()[0].page_content

prompt_extract = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template("""
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the 
    following keys: `role`, `experience`, `skills`, and `description`.
    Only return the valid JSON. No explanation or additional text.
    ### VALID JSON (NO PREAMBLE):
    """)
])

chain_extract = prompt_extract | llm
response = chain_extract.invoke({"page_data": page_data})

import json
parsed = json.loads(response.content)
job = parsed[0] if isinstance(parsed, list) else parsed
print("📌 Extracted Job Info:\n", job)

df = pd.read_csv("my_portfolio.csv") 
print("📚 Portfolio Data:\n", df.head())

client = chromadb.PersistentClient(path='vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if collection.count() == 0:
    for _, row in df.iterrows():
        collection.add(
            documents=[row["Techstack"]],
            metadatas=[{"link": row["Links"]}],
            ids=[str(uuid.uuid4())]
        )

results = collection.query(query_texts=[" ".join(job['skills'])], n_results=2)
matched_links = [item['link'] for item in results['metadatas'][0]]
print("🔗 Matching Portfolios:\n", matched_links)

email_prompt = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template("""
    You're an AI assistant helping a candidate write a cold email to apply for a job.

    Job Role: {role}  
    Experience Required: {experience}  
    Skills Required: {skills}  
    Description: {description}  
    Portfolio Links: {links}

    Write a cold email with a subject line. Be concise, confident, and professional.
    """)
])

email_chain = email_prompt | llm
email_output = email_chain.invoke({
    "role": job['role'],
    "experience": job['experience'],
    "skills": job['skills'],
    "description": job['description'],
    "links": ", ".join(matched_links)
})

print("📧 Cold Email Generated:\n\n", email_output.content)



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


📌 Extracted Job Info:
 {'role': 'Software Engineer', 'experience': '4+ years', 'skills': ['Java', 'Kotlin', 'Go', 'Scala', 'Python', 'Oracle', 'Postgres', 'DynamoDB', 'Cassandra', 'AWS', 'GAE', 'Azure', 'Cloud architecture patterns'], 'description': 'Build and ship features and capabilities daily in highly scalable, cross-geo distributed environment. Be part of an amazing open and collaborative work environment with other experienced engineers, architects, product managers, and designers. Review code with best practices of readability, testing patterns, documentation, reliability, security, and performance considerations in mind. Mentor and level up the skills of your teammates by sharing your expertise in formal and informal knowledge sharing sessions. Ensure full visibility, error reporting, and monitoring of high performing backend services. Participate in Agile software development including daily stand-ups, sprint planning, team retrospectives, show and tell demo sessions.'}
📚 Por